#### Rex Gayas
#### Week 8 Exercise 8.2 Spring 2024
#### DSC400-T301 Big Data, Technology, and Algo (2245-1)
#### Stream Processing with Spark

In [1]:
import pandas as pd
import numpy as np

base_github_repo_url = 'https://raw.githubusercontent.com/bellevue-university/dsc400/main'
family_csv_url = base_github_repo_url + '/data/stardew/family.csv'
friends_csv_url = base_github_repo_url + '/data/stardew/friends.csv'
gifts_csv_url = base_github_repo_url + '/data/stardew/gifts.csv'
villagers_csv_url = base_github_repo_url + '/data/stardew/villagers.csv'

pd_df_family = pd.read_csv(family_csv_url, index_col='id')
pd_df_gifts = pd.read_csv(gifts_csv_url, index_col='id')
pd_df_friends = pd.read_csv(friends_csv_url, index_col='id')
pd_df_villagers = pd.read_csv(villagers_csv_url, index_col='id')

pd_df_villagers['birthday'] = pd_df_villagers['birthday'].replace(np.nan, 'Unknown')
pd_df_villagers['address'] = pd_df_villagers['address'].replace(np.nan, 'Unknown')

In [2]:
from pathlib import Path
import os
import shutil

current_working_dir = Path(os.getcwd())
spark_warehouse_dir = current_working_dir.joinpath('spark-warehouse')
if spark_warehouse_dir.exists():
    shutil.rmtree(spark_warehouse_dir)

In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a52e3e65e09f464dece7e16db22e9bbb9ee6f536e6c2951f150b6d4766ac94c4
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Test App") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext


In [6]:
# Simple test to see if the SparkContext is working
rdd = sc.parallelize([1, 2, 3, 4, 5])
result = rdd.map(lambda x: x * x).collect()
print(result)


[1, 4, 9, 16, 25]


#### Assignment 8.1

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType

# Initialize Spark Session
spark = SparkSession.builder.appName("DSC 400 Assignment 8").getOrCreate()

# Schema definition
villager_schema = StructType([
    StructField("name", StringType(), True),
    StructField("birthday", StringType(), True),
    StructField("address", StringType(), True),
    StructField("is_marriable", BooleanType(), True),
    StructField("img_url", StringType(), True)
])

df_villagers = spark.createDataFrame(pd_df_villagers, villager_schema)
df_villagers.createOrReplaceTempView("villagers")
df_villagers.printSchema()
df_villagers.show()

df_family = spark.createDataFrame(pd_df_family)
df_family.createOrReplaceTempView("family")
df_family.printSchema()
df_family.show()

df_gifts = spark.createDataFrame(pd_df_gifts)
df_gifts.createOrReplaceTempView("gifts")
df_gifts.printSchema()
df_gifts.show()

df_friends = spark.createDataFrame(pd_df_friends)
df_friends.createOrReplaceTempView("friends")
df_friends.printSchema()
df_friends.show()


root
 |-- name: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- address: string (nullable = true)
 |-- is_marriable: boolean (nullable = true)
 |-- img_url: string (nullable = true)

+---------+---------+--------------------+------------+--------------------+
|     name| birthday|             address|is_marriable|             img_url|
+---------+---------+--------------------+------------+--------------------+
|  Abigail|  Fall 13|Pierre's General ...|        true|https://stardewva...|
|     Alex|Summer 13|        1 River Road|        true|https://stardewva...|
|   Birdie|  Unknown|  Hut on Island West|       false|https://stardewva...|
|  Bouncer|  Unknown|               Oasis|       false|https://stardewva...|
| Caroline| Winter 7|Pierre's General ...|       false|https://stardewva...|
|    Clint|Winter 26|          Blacksmith|       false|https://stardewva...|
|Demetrius|Summer 19|    24 Mountain Road|       false|https://stardewva...|
|    Dwarf|Summer 22|    

Started by importing the necessary modules and initializing a Spark session. Using this session, I converted Pandas dataframes for family, gifts, friends, and villagers into Spark DataFrames. For each Spark DataFrame created, I printed the schema to understand the structure of the data, which includes column names and data types. This helps verify that the data was imported correctly into Spark.

Used the .show() method to display the contents of each DataFrame, ensuring that the data was loaded as expected and to provide a quick snapshot of what each DataFrame contains. For the villagers DataFrame, I defined an explicit schema to help Spark better interpret the data structure, especially when the automatic inference might not accurately capture the data types or nullable status of the columns.

##### Assignment 8.2

In [8]:
# Create a temporary 'family' view using the 'df_family' DataFrame
df_family.createOrReplaceTempView("family")

# Create a temporary 'friends' view using the 'df_friends' DataFrame
df_friends.createOrReplaceTempView("friends")

# Create a temporary 'gifts' view using the 'df_gifts' DataFrame
df_gifts.createOrReplaceTempView("gifts")

# Create a temporary 'villagers' view using the 'df_villagers' DataFrame
df_villagers.createOrReplaceTempView("villagers")


In [9]:
# Verify 'family' view by displaying the first 20 rows
print("Displaying data from the family view:")
spark.sql("SELECT * FROM family").show()

# Verify 'friends' view by displaying the first 20 rows
print("Displaying data from the friends view:")
spark.sql("SELECT * FROM friends").show()

# Verify 'gifts' view by displaying the first 20 rows
print("Displaying data from the gifts view:")
spark.sql("SELECT * FROM gifts").show()

# Verify 'villagers' view by displaying the first 20 rows
print("Displaying data from the villagers view:")
spark.sql("SELECT * FROM villagers").show()


Displaying data from the family view:
+-----------+----------------+------------+
|villager_id|family_member_id|relationship|
+-----------+----------------+------------+
|    Abigail|          Pierre|      Father|
|    Abigail|        Caroline|      Mother|
|       Alex|          Evelyn| Grandmother|
|       Alex|          George| Grandfather|
|   Caroline|          Pierre|     Husband|
|   Caroline|         Abigail|    Daughter|
|  Demetrius|           Robin|        Wife|
|  Demetrius|            Maru|    Daughter|
|  Demetrius|       Sebastian|    Step-son|
|      Emily|           Haley|      Sister|
|     Evelyn|          George|     Husband|
|     Evelyn|            Alex|    Grandson|
|     George|          Evelyn|        Wife|
|     George|            Alex|    Grandson|
|      Haley|           Emily|      Sister|
|        Jas|          Marnie|        Aunt|
|        Jas|           Shane|   Godfather|
|       Jodi|            Kent|     Husband|
|       Jodi|             Sam|        

Created DataFrames in PySpark. Using the createOrReplaceTempView method, each DataFrame is registered as a temporary view. These views allowed me to run SQL queries directly on the DataFrame data as if they were tables in a database. Once the views are created, you I used standard SQL queries to interact with the data. This is particularly useful for filtering, aggregating, or joining data in complex ways that are sometimes more intuitive in SQL.

#### Assignment 8.3

##### Assignment 8.3a

In [10]:
# Run SQL query to return a list of Sebastian's friends
sebastians_friends_query = """
SELECT friend_id
FROM friends
WHERE villager_id = 'Sebastian'
"""
result = spark.sql(sebastians_friends_query)
result.show()


+---------+
|friend_id|
+---------+
|  Abigail|
|      Sam|
+---------+



##### Assignment 8.3b

In [11]:
# Group the 'friends' table by 'villager_id' and perform a count of number of friends for each villager
friends_count_query = """
SELECT villager_id, COUNT(friend_id) as number_of_friends
FROM friends
GROUP BY villager_id
"""
result_grouped = spark.sql(friends_count_query)
result_grouped.show()


+-----------+-----------------+
|villager_id|number_of_friends|
+-----------+-----------------+
|      Clint|                1|
|       Jodi|                1|
|     Harvey|                1|
|       Alex|                1|
|    Elliott|                2|
|      Emily|                1|
|       Leah|                1|
|        Gus|                1|
|        Leo|                3|
|        Jas|                2|
|      Haley|                1|
|    Abigail|                2|
|   Caroline|                1|
|        Gil|                1|
|      Penny|                2|
|      Sandy|                1|
|      Linus|                2|
|     Wizard|                1|
|      Lewis|                1|
|      Willy|                1|
+-----------+-----------------+
only showing top 20 rows




Used a SELECT statement to retrieve specific information from a view. Fetched all friends of a particular character in a game or dataset by filtering based on a column value (like villager_id). Used SQL to group data by certain identifiers and performing calculations on each group, such as counting the number of friends each villager has. Each SQL query is executed using the spark.sql() method and results were exhibited via show().
